In [1]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print("role" + role)

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'ic-fulltraining'

rolearn:aws:iam::334381301451:role/service-role/AmazonSageMaker-ExecutionRole-20190712T131802


In [2]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification')
print (training_image)



433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:1


In [3]:
# Four channels: train, validation, train_lst, and validation_lst
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)
print(s3train)
print(s3validation)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)
print(s3_output_location)




s3://sagemaker-us-west-2-334381301451/ic-fulltraining/train/
s3://sagemaker-us-west-2-334381301451/ic-fulltraining/validation/
s3://sagemaker-us-west-2-334381301451/ic-fulltraining/output


In [4]:
ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p2.xlarge',
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode = 'File',
                                         output_path = s3_output_location,
                                         sagemaker_session=sess)


In [5]:
ic.set_hyperparameters(num_layers=18,
                             image_shape = "1,350,360",
                             num_classes=2,
                             num_training_samples=1200,
                             mini_batch_size=10,
                             epochs=5,
                             learning_rate=0.01,
                             top_k=2,
                             precision_dtype='float32')

In [6]:
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}


In [7]:

ic.fit(inputs=data_channels, logs=True)



2019-08-06 01:46:55 Starting - Starting the training job...
2019-08-06 01:46:57 Starting - Launching requested ML instances......
2019-08-06 01:48:00 Starting - Preparing the instances for training.........
2019-08-06 01:49:53 Downloading - Downloading input data
2019-08-06 01:49:53 Training - Downloading the training image......
2019-08-06 01:50:52 Training - Training image download completed. Training in progress..
Docker entrypoint called with argument(s): train
[08/06/2019 01:50:54 INFO 139867713042240] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[08/06/2019 01:50:54 INFO 139867713042240]

[08/06/2019 01:52:43 INFO 139867713042240] Epoch[3] Train-accuracy=1.000000
[08/06/2019 01:52:43 INFO 139867713042240] Epoch[3] Train-top_k_accuracy_2=1.000000
[08/06/2019 01:52:43 INFO 139867713042240] Epoch[3] Time cost=22.736
[08/06/2019 01:52:46 INFO 139867713042240] Epoch[3] Validation-accuracy=1.000000
[08/06/2019 01:52:51 INFO 139867713042240] Epoch[4] Batch [20]#011Speed: 47.152 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[08/06/2019 01:52:54 INFO 139867713042240] Epoch[4] Batch [40]#011Speed: 50.020 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[08/06/2019 01:52:58 INFO 139867713042240] Epoch[4] Batch [60]#011Speed: 50.961 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[08/06/2019 01:53:02 INFO 139867713042240] Epoch[4] Batch [80]#011Speed: 51.548 samples/sec#011accuracy=1.000000#011top_k_accuracy_2=1.000000
[08/06/2019 01:53:06 INFO 139867713042240] Epoch[4] Batch [100]#011Speed: 51.879 samples/sec#011accuracy=1.000000#011top_k

In [14]:
import boto3
optimized_ic = ic
try:
    ic.create_model()._neo_image_account(boto3.Session().region_name)
except:
    print('Neo is not currently supported in', boto3.Session().region_name)
else:
    output_path = '/'.join(ic.output_path.split('/')[:-1])
    optimized_ic = ic.compile_model(target_instance_family='ml_m4', 
                                input_shape={'data':[1, 1, 350, 360]},  # Batch size 1, 3 channels, 224x224 Images.
                                output_path=output_path,
                                framework='mxnet', framework_version='1.2.1')
    optimized_ic.image = get_image_uri(sess.boto_region_name, 'image-classification-neo', repo_version="latest")
    optimized_ic.name = 'deployed-image-classification'



??......!

In [15]:
ic_classifier = optimized_ic.deploy(initial_instance_count = 1,
                                          instance_type = 'ml.t2.medium')



Using already existing model: deployed-image-classificationml-t2


----------------------------------------------------------------------------------------------------------------------------!

In [16]:
file_lst =[]
file_lst.append('cutted_img_0007.jpg')
file_lst.append('cutted_img_0008.jpg')
file_lst.append('cutted_img_0009.jpg')
file_lst.append('cutted_img_0010.jpg')
file_lst.append('cutted_img_6.jpg')
file_lst.append('cutted_img_7.jpg')
file_lst.append('cutted_img_8.jpg')
file_lst.append('cutted_img_9.jpg')



In [17]:
import json
import numpy as np

for file_name in file_lst: 
    with open(file_name, 'rb') as f:
        payload = f.read()
        payload = bytearray(payload)
        ic_classifier.content_type = 'application/x-image'
        result = json.loads(ic_classifier.pr,edict(payload))
        print(result)
        index = np.argmax(result)
        object_categories = ['abnormal','normal']
        print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))
        print()
        print()

# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index


[1.0, 7.775905693749768e-21]
Result: label - abnormal, probability - 1.0


[1.0, 6.749181814026387e-20]
Result: label - abnormal, probability - 1.0


[1.0, 2.8317375145436087e-21]
Result: label - abnormal, probability - 1.0


[1.0, 9.847611041837937e-20]
Result: label - abnormal, probability - 1.0


[0.08254571259021759, 0.9174543619155884]
Result: label - normal, probability - 0.9174543619155884


[0.047843821346759796, 0.952156126499176]
Result: label - normal, probability - 0.952156126499176


[0.115699402987957, 0.8843005895614624]
Result: label - normal, probability - 0.8843005895614624


[0.993401825428009, 0.006598117295652628]
Result: label - abnormal, probability - 0.993401825428009


